In [1]:
!git clone https://github.com/nkt780426/concat.git

Cloning into 'Bachelor-s-Project'...
remote: Enumerating objects: 566, done.
remote: Counting objects: 100% (158/158), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 566 (delta 46), reused 117 (delta 33), pack-reused 408 (from 1)
Receiving objects: 100% (566/566), 77.12 MiB | 47.77 MiB/s, done.
Resolving deltas: 100% (47/47), done.


In [2]:
!mv concat/going_modular .

In [3]:
!ls -la

total 52
drwxr-xr-x 4 root root  4096 Dec 20 13:58 .
drwxr-xr-x 6 root root  4096 Dec 20 13:58 ..
drwxr-xr-x 5 root root  4096 Dec 20 13:58 Bachelor-s-Project
drwxr-xr-x 8 root root  4096 Dec 20 13:58 going_modular
---------- 1 root root 32885 Dec 20 13:58 __notebook__.ipynb


In [4]:
import warnings
warnings.filterwarnings("ignore")

import torch
from torchvision import transforms
from torch.utils.data import DataLoader
from torch.optim import Adam
from torch.optim.lr_scheduler import CosineAnnealingWarmRestarts
import albumentations as A

from going_modular.model.backbone.iresnet import iresnet18
from going_modular.model.TripletFaceRecognition import TripletNet
from going_modular.dataloader.triplet import TripletDataset
from going_modular.loss.TripletLoss import TripletLoss
from going_modular.train_eval.triplet.train import fit
from going_modular.dataloader.triplet import CustomExrDataset
from going_modular.utils.MultiMetricEarlyStopping import MultiMetricEarlyStopping
from going_modular.utils.ModelCheckPoint import ModelCheckpoint
from going_modular.utils.transforms import RandomResizedCropRect, GaussianNoise

import os

device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)
os.environ["PYTORCH_CUDA_ALLOC_CONF"] = "expandable_segments:True"

# Đặt seed toàn cục
seed = 42
torch.manual_seed(seed)

CONFIGURATION = {
    # Thư mục
    'type': 'albedo',
    'train_dir': '/kaggle/input/hoangvn-3dfacerecognition/Albedo/train',
    'test_dir': '/kaggle/input/hoangvn-3dfacerecognition/Albedo/test',
    'checkpoint_dir': './checkpoint/new/',
    
    # Cấu hình train
    'epochs': 399,
    'num_workers': 4,
    'batch_size': 16,
    'image_size': 256,
    'embedding_size': 512,
    
    'start_lr': 1e-4,
    'weight_decay': 5e-4,
    'momentum': 0.9,
    'alpha': 0.9,
    
    # triplet
    'margin': 1.,
}

train_transform = A.Compose([
    RandomResizedCropRect(256),
    GaussianNoise(),
])

test_transform = A.Compose([
    A.Resize(height=CONFIGURATION['image_size'], width=CONFIGURATION['image_size'])
])

triplet_train_dataset = TripletDataset(data_dir=CONFIGURATION['train_dir'], type=CONFIGURATION['albedo'], transform=train_transform, train=True)
triplet_test_dataset = TripletDataset(data_dir=CONFIGURATION['train_dir'], type=CONFIGURATION['albedo'], transform=test_transform, train=False)

triplet_train_loader = DataLoader(
    triplet_train_dataset, 
    batch_size=CONFIGURATION['batch_size'], 
    shuffle=True,
    num_workers=4,
    pin_memory=True
)

triplet_test_loader = DataLoader(
    triplet_test_dataset, 
    batch_size=CONFIGURATION['batch_size'], 
    shuffle=False,
    num_workers=4,
    pin_memory=True
)

roc_train_dataset = CustomExrDataset(CONFIGURATION['train_dir'], transform=train_transform, type=CONFIGURATION['type'])
roc_train_loader = DataLoader(
    roc_train_dataset,
    num_workers=4,
    batch_size=CONFIGURATION['batch_size'],
)

roc_test_dataset = CustomExrDataset(CONFIGURATION['test_dir'], transform=test_transform, type=CONFIGURATION['type'])
roc_test_loader = DataLoader(
    roc_test_dataset,
    num_workers=4,
    batch_size=CONFIGURATION['batch_size'],
)

backbone = iresnet18(num_classes=CONFIGURATION['embedding_size']).to(device)
model = TripletNet(backbone).to(device)
criterion = TripletLoss(CONFIGURATION['margin'])
optimizer = Adam(model.parameters(), lr=CONFIGURATION['start_lr'])
scheduler = CosineAnnealingWarmRestarts(optimizer, T_0=50, T_mult=1, eta_min=1e-6)

earlystop_dir = os.path.abspath(CONFIGURATION['checkpoint_dir'] + CONFIGURATION['type'] + '/models')

early_stopping = MultiMetricEarlyStopping(
    monitor_keys=['cosine_auc', 'euclidean_auc'],
    patience=1000,
    mode='max',
    verbose=0,
    save_dir=earlystop_dir,
    start_from_epoch=50
)      

checkpoint_path = os.path.abspath(CONFIGURATION['checkpoint_dir'] + CONFIGURATION['type'] + '/models/checkpoint.pth')
modle_checkpoint = ModelCheckpoint(filepath=checkpoint_path, verbose=0)

cuda


In [5]:
fit(
    conf=CONFIGURATION,
    start_epoch=0,
    model=model,
    triplet_train_loader=triplet_train_loader, 
    triplet_test_loader=triplet_test_loader, 
    criterion=criterion,
    optimizer=optimizer, 
    scheduler=scheduler, 
    epochs=CONFIGURATION['epochs'], 
    device=device, 
    roc_train_loader=roc_train_loader, 
    roc_test_loader=roc_test_loader,
    early_stopping=early_stopping,
    model_checkpoint=modle_checkpoint
)

Epoch 1:
	train: loss: 1.1096 | auc_cos: 0.6938 | auc_eu: 0.6831
	test: loss: 0.8010 | auc_cos: 0.7773 | auc_eu: 0.7556
Epoch 2:
	train: loss: 0.7443 | auc_cos: 0.6982 | auc_eu: 0.6908
	test: loss: 0.7042 | auc_cos: 0.7727 | auc_eu: 0.7572
Epoch 3:
	train: loss: 0.6784 | auc_cos: 0.7316 | auc_eu: 0.7312
	test: loss: 0.5199 | auc_cos: 0.8018 | auc_eu: 0.7986
Epoch 4:
	train: loss: 0.5855 | auc_cos: 0.7413 | auc_eu: 0.7297
	test: loss: 0.8102 | auc_cos: 0.8160 | auc_eu: 0.7996
Epoch 5:
	train: loss: 0.5967 | auc_cos: 0.7715 | auc_eu: 0.7712
	test: loss: 0.4284 | auc_cos: 0.7999 | auc_eu: 0.8023
Epoch 6:
	train: loss: 0.5659 | auc_cos: 0.7672 | auc_eu: 0.7612
	test: loss: 0.4982 | auc_cos: 0.8098 | auc_eu: 0.8070
Epoch 7:
	train: loss: 0.5648 | auc_cos: 0.7838 | auc_eu: 0.7836
	test: loss: 0.3765 | auc_cos: 0.8338 | auc_eu: 0.8363
Epoch 8:
	train: loss: 0.4875 | auc_cos: 0.8175 | auc_eu: 0.8115
	test: loss: 0.3606 | auc_cos: 0.8556 | auc_eu: 0.8555
Epoch 9:
	train: loss: 0.4703 | auc_cos: